In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys

In [2]:
# krx에서 상장기업 리스트 가져오기
stock_df = pd.read_html( 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]
stock_df['종목코드'] = stock_df['종목코드'].map(lambda x: f'{x:0>6}')
corp_list = stock_df[['회사명','종목코드','업종','지역']]
corp_list

,회사명,종목코드,업종,지역
0,DSR,155660,1차 비철금속 제조업,부산광역시
1,GS,078930,기타 금융업,서울특별시
2,GS글로벌,001250,상품 종합 도매업,서울특별시
3,HDC현대산업개발,294870,건물 건설업,서울특별시
4,LG이노텍,011070,전자부품 제조업,서울특별시
...,...,...,...,...
2347,지앤이헬스케어,299480,기타 섬유제품 제조업,경기도
2348,코셋,189350,통신 및 방송 장비 제조업,광주광역시
2349,툴젠,199800,자연과학 및 공학 연구개발업,서울특별시
2350,한국미라클피플사,331660,기타 화학제품 제조업,경기도


In [3]:
corp = corp_list['회사명'].values

In [14]:
driver=webdriver.Chrome("chromedriver.exe")
driver.get('https://www.jobplanet.co.kr/search?query='+corp[0]+'&category=&_rs_con=search&_rs_act=index&_rs_element=main_search_bar')

In [15]:
# 팝업제거
icon = driver.find_element_by_class_name("tit")
icon.click()
time.sleep(0.2)

In [16]:
close = driver.find_elements_by_class_name("btn_close_x_ty1")

In [17]:
close[1].click()

In [19]:
# 페이지소스 가져오기
html = driver.page_source
soup = BeautifulSoup(html)

# 복지 및 급여
welfare = soup.find_all(class_='txt_point')[0].text
# 업무와 삶의 균형
balance = soup.find_all(class_='txt_point')[1].text
# 사내문화
culture = soup.find_all(class_='txt_point')[2].text
# 승진 기회 및 가능성
possibility = soup.find_all(class_='txt_point')[3].text
# 경영진
executive = soup.find_all(class_='txt_point')[4].text
# 총 만족도
total = soup.find_all(class_='rate_point')[0].text

In [24]:
name = soup.find_all(class_='word')[0].text

In [25]:
dict = {'회사명':[corp[0]],'네임':[name],'복지':[welfare],'워라벨':[balance],'사내문화':[culture],'가능성':[possibility],'경영진':[executive],'총만족도':[total]}

In [27]:
test = pd.DataFrame(dict)

In [30]:
test

,회사명,네임,복지,워라벨,사내문화,가능성,경영진,총만족도
0,DSR,DSR,3.1,2.1,2.3,2.5,2.2,2.6


In [29]:
corp_list.append(test)

,회사명,종목코드,업종,지역,네임,복지,워라벨,사내문화,가능성,경영진,총만족도
0,DSR,155660,1차 비철금속 제조업,부산광역시,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GS,078930,기타 금융업,서울특별시,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GS글로벌,001250,상품 종합 도매업,서울특별시,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HDC현대산업개발,294870,건물 건설업,서울특별시,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LG이노텍,011070,전자부품 제조업,서울특별시,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2348,코셋,189350,통신 및 방송 장비 제조업,광주광역시,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2349,툴젠,199800,자연과학 및 공학 연구개발업,서울특별시,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2350,한국미라클피플사,331660,기타 화학제품 제조업,경기도,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2351,휴벡셀,212310,의료용 기기 제조업,경기도,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
pd.merge(corp_list,test)

,회사명,종목코드,업종,지역,네임,복지,워라벨,사내문화,가능성,경영진,총만족도
0,DSR,155660,1차 비철금속 제조업,부산광역시,DSR,3.1,2.1,2.3,2.5,2.2,2.6
